In [ ]:
# Binning if needed in other classifiers
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/graph_train.csv')
y = pd.cut(df['edge_cross_norm'], bins=[-1,-1e-10,1]).to_numpy()
df = df.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0'],axis=1)
list_columns = list(df)

for col in list_columns:
    print(col)
    if col!= 'edge_cross_norm' and col!= 'is_bridge':
        df[col] = pd.qcut(df[col],q=5)

X = df.to_numpy()
df.head()


edge_betweenness
stress
is_bridge
diff_stress
diff_cross
diff_edgelength
exp_factor_norm
sum_neighbour_deg_norm
max_neighbour_deg_norm
grad_diff


,edge_betweenness,stress,is_bridge,diff_stress,diff_cross,diff_edgelength,exp_factor_norm,sum_neighbour_deg_norm,max_neighbour_deg_norm,grad_diff
0,"(0.0451, 0.0702]","(0.0818, 0.684]",False,"(0.357, 1.41]","(-2.0, 0.0]","(-0.00824, -0.00378]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
1,"(-0.000785, 0.0219]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
2,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
3,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(2.0, 39.0]","(-0.516, -0.00824]","(0.109, 0.607]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
4,"(0.0702, 0.545]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(-2.0, 0.0]","(0.00165, 0.266]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"


In [2]:
# XGBoost data preparation 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df2 = pd.read_csv('../data/graph_train.csv')
df2['is_bridge'] = df2['is_bridge'].astype(float)
yn = (df2['edge_cross_norm']>0).to_numpy(dtype=int)
df2 = df2.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0','diff_cross'],axis=1)
Xn = df2.to_numpy(dtype=float)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn,yn,shuffle=True)

In [3]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(Xn_train,yn_train,verbose = True)
y_pred = xgb.predict(Xn_test)
pos = np.sum([1 if y == 1 else 0 for y in yn_test])
print('Amount of better:',pos)
print('Amount of neutral or worse:',len(yn_test)-pos)
print('Accuracy in test:',(sum([1 if yn_test[i]==y_pred[i] else 0 for i in range(len(y_pred))]))/len(yn_test))

[23:02:50] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Amount of better: 1335
Amount of neutral or worse: 3698
Accuracy in test: 0.7440890125173852


In [3]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',silent=True, nthread=1)

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [6, 8, 10]
        }

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(Xn_train,yn_train), verbose=3 )
grid.fit(Xn_train, yn_train)

y_test = grid.best_estimator_.predict_proba(Xn_test)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits


c:\Users\natha\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:38:24] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:38:25] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [7]:
print(y_test)
y_pred = [1 if y_test[i][1]>0.5 else 0 for i in range(len(y_test))]
acc = np.sum([1 if y_pred[i]==yn_test[i] else 0 for i in range(len(y_test))])/len(y_test)
print(acc)
# Grid search useless, same results

[[0.41215986 0.58784014]
 [0.57166505 0.42833498]
 [0.9942242  0.00577581]
 ...
 [0.18812478 0.8118752 ]
 [0.6319574  0.36804262]
 [0.8036338  0.19636619]]
0.7401152394198292
